In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
from PIL import Image, ImageFilter
import cv2
import random
import roboticstoolbox as rtb
from roboticstoolbox import DistanceTransformPlanner
import math
from spatialmath.base import *
from math import sin, cos, atan2, radians
from random import uniform
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from scipy.interpolate import CubicSpline
from numpy.typing import ArrayLike
%matplotlib qt



In [3]:
def processMap(map_path, scale_percent):
    img=np.array(Image.open(map_path).convert("L"))
    imgColor=np.array(Image.open(map_path))

    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img=cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    imgColor=cv2.resize(imgColor, dim, interpolation = cv2.INTER_AREA)

    imgSharp=cv2.filter2D(img,-1,np.array([[0, -1, 0],[-1, 5,-1],[0, -1, 0]])) 
    imgdia=np.zeros((np.shape(img)))
    imgdia[img==255]=0
    imgdia[img<255]=1
    imgdia=cv2.dilate(imgdia,np.ones((1,1),np.uint8))

    binMap=np.zeros((np.shape(img)))
    binMap[imgSharp < 255] = 0
    binMap[imgSharp==255] = 1
    
    return img, imgColor, imgdia, binMap

color2speed = {81:5, 114:6, 177:7, 219:10, 118:15, 156:17, 159:20, 171:30, 155:40, 92:50, 30:60, 61:70, 92:80, 98:100}


map_path2 = '../../results/berlinCloseup.png'
img2, imgColor2, imgdia2, binMap2=processMap(map_path2,2)

In [4]:
# f,ax = plt.subplots(1,2)
# ax[0].imshow(binMap, cmap='gray')
# ax[1].imshow(binMap2 , cmap='gray')
# ax[1].set_xlim([-600,600]), ax[1].set_ylim([-600,600])
# plt.show()

In [5]:
img, imgColor, imgdia, binMap=processMap('../../results/berlin.png', 40)
s=(905,68)
g=(1153,395)
dx = DistanceTransformPlanner(binMap, goal=g, distance="euclidean")
dx.plan()
path = dx.query(start=s)
# plt.plot(x,y,'.')



[[ 905   68]
 [ 906   69]
 [ 907   70]
 [ 907   71]
 [ 908   72]
 [ 908   73]
 [ 909   74]
 [ 910   75]
 [ 910   76]
 [ 911   77]
 [ 911   78]
 [ 912   79]
 [ 913   80]
 [ 913   81]
 [ 914   82]
 [ 915   83]
 [ 915   84]
 [ 916   85]
 [ 917   86]
 [ 918   87]
 [ 919   88]
 [ 919   89]
 [ 920   90]
 [ 920   91]
 [ 921   92]
 [ 922   93]
 [ 922   94]
 [ 923   95]
 [ 924   96]
 [ 924   97]
 [ 925   98]
 [ 926   99]
 [ 927  100]
 [ 927  101]
 [ 928  102]
 [ 929  103]
 [ 929  104]
 [ 930  105]
 [ 931  106]
 [ 932  107]
 [ 932  108]
 [ 933  109]
 [ 934  110]
 [ 934  111]
 [ 935  112]
 [ 936  113]
 [ 936  114]
 [ 937  115]
 [ 938  116]
 [ 939  117]
 [ 939  118]
 [ 940  119]
 [ 941  120]
 [ 941  121]
 [ 942  122]
 [ 943  123]
 [ 944  124]
 [ 944  125]
 [ 945  126]
 [ 946  127]
 [ 946  128]
 [ 947  129]
 [ 948  130]
 [ 949  131]
 [ 949  132]
 [ 950  133]
 [ 951  134]
 [ 951  135]
 [ 952  136]
 [ 953  137]
 [ 954  138]
 [ 954  139]
 [ 955  140]
 [ 956  141]
 [ 956  142]
 [ 957  143]
 [ 958  144]

In [109]:
def rescalePath(path, img, scale_percent):
    #convert path to image
    path_x, path_y = zip(*path)
    pathImg=np.zeros((np.shape(img)))
    pathImg[(path_y, path_x)]=1

    # scale down path image by given percentage 
    width = int(pathImg.shape[1] * scale_percent / 100)
    height = int(pathImg.shape[0] * scale_percent / 100)
    dim = (width, height)
    newImg= cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    pathImgRescaled=cv2.resize(pathImg, dim, interpolation = cv2.INTER_AREA)
    
    #identify new path from rescaled image 
    pathImgRescaled[pathImgRescaled>0]=1
    newYpath, newXpath=np.where(pathImgRescaled==1)
    return tuple(newXpath), tuple(newYpath), newImg 

scaleXpath, scaleYpath, scaleImg= rescalePath(path, img, 25)

# a,b = path_x, path_y
# plt.imshow(imgdia, cmap='gray')
# plt.plot(a,b,'r-')
# plt.show()


In [123]:
normalise_angle = lambda angle: atan2(sin(angle), cos(angle))

class Vehicle:
    def __init__(self, path_x, path_y, throttle, dt,\
        control_gain=20, softening_gain=2.0, yaw_rate_gain=0.10, steering_damp_gain=0.0, max_steer=np.deg2rad(45), \
        c_r: float=0.01, c_a: float=2.0, wheelbase=2.96, \
        overall_length=4.97, overall_width=1.964, rear_overhang=0.0, tyre_diameter=0.4826, \
        tyre_width=0.265, axle_track=1.7):
      
        self.k = control_gain
        self.k_soft = softening_gain
        self.k_yaw_rate = yaw_rate_gain
        self.k_damp_steer = steering_damp_gain
        self.max_steer = max_steer
        self.wheelbase = wheelbase

        self.px = path_x
        self.py = path_y
        self.pyaw=self.calculate_spline_yaw(self.px,self.py)

        self.x= path_x[0]
        self.y= path_y[0]
        self.yaw= self.pyaw[0]
        self.crosstrack_error = None
        self.target_id = 0

        self.v = 0.0
        self.delta = 0.0
        self.omega = 0.0
        self.throttle = throttle

        self.dt = dt
        self.c_r = c_r
        self.c_a = c_a

        self.rear_overhang=0.5 * (overall_length - self.wheelbase)
        rear_axle_to_front_bumper  = overall_length - rear_overhang
        centreline_to_wheel_centre = 0.5 * axle_track
        centreline_to_side         = 0.5 * overall_width
        vehicle_vertices = np.array([
            (-rear_overhang,              centreline_to_side),
            ( rear_axle_to_front_bumper,  centreline_to_side),
            ( rear_axle_to_front_bumper, -centreline_to_side),
            (-rear_overhang,             -centreline_to_side)
        ])
        half_tyre_width            = 0.5 * tyre_width
        centreline_to_inwards_rim  = centreline_to_wheel_centre - half_tyre_width
        centreline_to_outwards_rim = centreline_to_wheel_centre + half_tyre_width
        # Rear right wheel vertices
        wheel_vertices = np.array([
            (-tyre_diameter, -centreline_to_inwards_rim),
            ( tyre_diameter, -centreline_to_inwards_rim),
            ( tyre_diameter, -centreline_to_outwards_rim),
            (-tyre_diameter, -centreline_to_outwards_rim)
        ])
        self.outlines         = np.concatenate([vehicle_vertices, [vehicle_vertices[0]]])
        self.rear_right_wheel = np.concatenate([wheel_vertices,   [wheel_vertices[0]]])
        # Reflect the wheel vertices about the x-axis
        self.rear_left_wheel  = self.rear_right_wheel.copy()
        self.rear_left_wheel[:, 1] *= -1
        # Translate the wheel vertices to the front axle
        front_left_wheel  = self.rear_left_wheel.copy()
        front_right_wheel = self.rear_right_wheel.copy()
        front_left_wheel[:, 0]  += wheelbase 
        front_right_wheel[:, 0] += wheelbase
        get_face_centre = lambda vertices: np.array([
            0.5*(vertices[0][0] + vertices[2][0]),
            0.5*(vertices[0][1] + vertices[2][1])
        ])
        # Translate front wheels to origin
        self.fr_wheel_centre = get_face_centre(front_right_wheel)
        self.fl_wheel_centre = get_face_centre(front_left_wheel)
        self.fr_wheel_origin = front_right_wheel - self.fr_wheel_centre
        self.fl_wheel_origin = front_left_wheel - self.fl_wheel_centre
        
    
    
    def initialise_cubic_spline(self, x: ArrayLike, y: ArrayLike, ds: float, bc_type: str):

        distance = np.concatenate((np.zeros(1), np.cumsum(np.hypot(np.ediff1d(x), np.ediff1d(y)))))
        points = np.array([x, y]).T
        s = np.arange(0, distance[-1], ds)

        try:
            cs = CubicSpline(distance, points, bc_type=bc_type, axis=0, extrapolate=False)
            
        except ValueError as e:
            raise ValueError(f"{e} If you are getting a sequence error, do check if your input dataset contains consecutive duplicate(s).")

        return cs, s

    def calculate_spline_yaw(self, x: ArrayLike, y: ArrayLike, ds: float=0.05, bc_type: str='natural'):
        
        cs, s = self.initialise_cubic_spline(x, y, ds, bc_type)
        dx, dy = cs.derivative(1)(s).T
        return np.arctan2(dy, dx)
   
    def find_target_path_id(self, x, y, yaw):  
        # Calculate position of the front axle
        fx = x + self.wheelbase * cos(yaw)
        fy = y + self.wheelbase * sin(yaw)

        dx = fx - self.px    # Find the x-axis of the front axle relative to the path
        dy = fy - self.py    # Find the y-axis of the front axle relative to the path

        d = np.hypot(dx, dy) # Find the distance from the front axle to the path
        target_index = np.argmin(d) # Find the shortest distance in the array

        return target_index, dx[target_index], dy[target_index], d[target_index]

    def calculate_yaw_term(self, target_index, yaw):
        yaw_error = normalise_angle(self.pyaw[target_index] - yaw)

        return yaw_error

    def calculate_crosstrack_term(self, target_velocity, yaw, dx, dy, absolute_error):
        front_axle_vector = np.array([sin(yaw), -cos(yaw)])
        nearest_path_vector = np.array([dx, dy])
        crosstrack_error = np.sign(nearest_path_vector@front_axle_vector) * absolute_error

        crosstrack_steering_error = atan2((self.k * crosstrack_error), (self.k_soft + target_velocity))

        return crosstrack_steering_error, crosstrack_error

    def calculate_yaw_rate_term(self, target_velocity, steering_angle):
        yaw_rate_error = self.k_yaw_rate*(-target_velocity*sin(steering_angle))/self.wheelbase

        return yaw_rate_error

    def calculate_steering_delay_term(self, computed_steering_angle, previous_steering_angle):
        steering_delay_error = self.k_damp_steer*(computed_steering_angle - previous_steering_angle)

        return steering_delay_error

    def stanley_control(self, x, y, yaw, target_velocity, steering_angle):
        target_index, dx, dy, absolute_error = self.find_target_path_id(x, y, yaw)
        yaw_error = self.calculate_yaw_term(target_index, yaw)
        crosstrack_steering_error, crosstrack_error = self.calculate_crosstrack_term(target_velocity, yaw, dx, dy, absolute_error)
        yaw_rate_damping = self.calculate_yaw_rate_term(target_velocity, steering_angle)
        
        desired_steering_angle = yaw_error + crosstrack_steering_error + yaw_rate_damping

        # Constrains steering angle to the vehicle limits
        desired_steering_angle += self.calculate_steering_delay_term(desired_steering_angle, steering_angle)
        limited_steering_angle = np.clip(desired_steering_angle, -self.max_steer, self.max_steer)

        return limited_steering_angle, target_index, crosstrack_error
        
    def kinematic_model(self, x: float, y: float, yaw: float, velocity: float, throttle: float, steering_angle: float):
        # Compute the local velocity in the x-axis
        friction     = velocity * (self.c_r + self.c_a*velocity)
        new_velocity = velocity + self.dt*(throttle - friction)

        # Limit steering angle to physical vehicle limits
        steering_angle = -self.max_steer if steering_angle < -self.max_steer else self.max_steer if steering_angle > self.max_steer else steering_angle

        # Compute the angular velocity
        angular_velocity = velocity*np.tan(steering_angle) / self.wheelbase

        # Compute the final state using the discrete time model
        new_x   = x + velocity*np.cos(yaw)*self.dt
        new_y   = y + velocity*np.sin(yaw)*self.dt
        new_yaw = normalise_angle(yaw + angular_velocity*self.dt)
        
        return new_x, new_y, new_yaw, new_velocity, steering_angle, angular_velocity

    def get_rotation_matrix(_, angle: float) -> np.ndarray:
        cos_angle = cos(angle)
        sin_angle = sin(angle)

        return np.array([
            ( cos_angle, sin_angle),
            (-sin_angle, cos_angle)
        ])

    def transform(self, point: np.ndarray) -> np.ndarray:
        # Vector rotation
        point = point.dot(self.yaw_vector).T

        # Vector translation
        point[0, :] += self.x
        point[1, :] += self.y
        
        return point

    def plot_car(self, x: float, y: float, yaw: float, steer: float):
        self.x = x
        self.y = y
        
        # Rotation matrices
        self.yaw_vector = self.get_rotation_matrix(yaw)
        steer_vector    = self.get_rotation_matrix(steer)

        # Rotate the wheels about its position
        front_right_wheel  = self.fr_wheel_origin.copy()
        front_left_wheel   = self.fl_wheel_origin.copy()
        front_right_wheel  = front_right_wheel@steer_vector
        front_left_wheel   = front_left_wheel@steer_vector
        front_right_wheel += self.fr_wheel_centre
        front_left_wheel  += self.fl_wheel_centre

        outlines          = self.transform(self.outlines)
        rear_right_wheel  = self.transform(self.rear_right_wheel)
        rear_left_wheel   = self.transform(self.rear_left_wheel)
        front_right_wheel = self.transform(front_right_wheel)
        front_left_wheel  = self.transform(front_left_wheel)

        return outlines, front_right_wheel, rear_right_wheel, front_left_wheel, rear_left_wheel
    
    def drive(self):
        # throttle = 300 #uniform(50, 200)
        self.delta, self.target_id, self.crosstrack_error = self.stanley_control(self.x, self.y, self.yaw, self.v, self.delta)
        self.x, self.y, self.yaw, self.v, _, ang_vel = self.kinematic_model(self.x, self.y, self.yaw, self.v, self.throttle, self.delta)

        # print(f"Cross-track term: {self.crosstrack_error}{' '*10}", end="\r")
        
        velocity_x.append(self.v*np.cos(self.yaw)*self.dt)
        velocity_y.append(self.v*np.sin(self.yaw)*self.dt)
        angular_velocity.append(ang_vel)

class Fargs:

    def __init__(self, ax, map_size_x, map_size_y, car, car_outline, front_right_wheel, front_left_wheel, rear_right_wheel, rear_left_wheel, rear_axle, annotation, target):

        self.ax                = ax
        self.map_size_x        = map_size_x
        self.map_size_y        = map_size_y
        self.car               = car
        self.car_outline       = car_outline
        self.front_right_wheel = front_right_wheel
        self.front_left_wheel  = front_left_wheel
        self.rear_right_wheel  = rear_right_wheel
        self.rear_left_wheel   = rear_left_wheel
        self.rear_axle         = rear_axle
        self.annotation        = annotation
        self.target            = target

def animate(frame, fargs):

    ax                = fargs.ax
    map_size_x        = fargs.map_size_x
    map_size_y        = fargs.map_size_y
    car               = fargs.car
    car_outline       = fargs.car_outline
    front_right_wheel = fargs.front_right_wheel
    front_left_wheel  = fargs.front_left_wheel
    rear_right_wheel  = fargs.rear_right_wheel
    rear_left_wheel   = fargs.rear_left_wheel
    rear_axle         = fargs.rear_axle
    annotation        = fargs.annotation
    target            = fargs.target

    # Camera tracks car
    ax.set_xlim(car.x - map_size_x, car.x + map_size_x)
    ax.set_ylim(car.y - map_size_y, car.y + map_size_y)

    # Drive and draw car
    if (car.px[car.target_id], car.py[car.target_id]) !=(car.px[-1], car.py[-1]):
        car.drive()
        outline_plot, fr_plot, rr_plot, fl_plot, rl_plot = car.plot_car(car.x, car.y, car.yaw, car.delta)
        car_outline.set_data(*outline_plot)
        front_right_wheel.set_data(*fr_plot)
        rear_right_wheel.set_data(*rr_plot)
        front_left_wheel.set_data(*fl_plot)
        rear_left_wheel.set_data(*rl_plot)
        rear_axle.set_data(car.x, car.y)
    else:
        car.v=0
    
    # Show car's target
    target.set_data(car.px[car.target_id], car.py[car.target_id])

    # Annotate car's coordinate above car
    annotation.set_text(f'{car.x:.1f}, {car.y:.1f}')
    annotation.set_position((car.x, car.y + 5))

    plt.title(f'{car.dt*frame:.2f}s', loc='right')
    plt.xlabel(f'Speed: {car.v:.2f} m/s', loc='left')
    # plt.savefig(f'image/visualisation_{frame:03}.png', dpi=300)

    return car_outline, front_right_wheel, rear_right_wheel, front_left_wheel, rear_left_wheel, rear_axle, target,

# Initial state
# x     =  s[0]
# y     = s[1]
# yaw   = (7*np.pi) / 4
# steer = np.deg2rad(25)
velocity_x=[]
velocity_y=[]
angular_velocity=[]
# Simulation Parameters
fps = 25
dt = 1/fps
map_size_x = 70
map_size_y = 40
frames = 2000
loop = False
car  = Vehicle(scaleXpath, scaleYpath,100, dt)
interval = car.dt * 10**3

fig = plt.figure()
ax = plt.axes()
ax.set_aspect('equal')

ax.plot(scaleYpath, scaleYpath, '--', color='gold')
ax.imshow(scaleImg)

empty              = ([], [])
target,            = ax.plot(*empty, '+r')
car_outline,       = ax.plot(*empty, color='black')
front_right_wheel, = ax.plot(*empty, color='black')
rear_right_wheel,  = ax.plot(*empty, color='black')
front_left_wheel,  = ax.plot(*empty, color='black')
rear_left_wheel,   = ax.plot(*empty, color='black')
rear_axle,         = ax.plot(car.x, car.y, '+', color='black', markersize=2)
annotation         = ax.annotate(f'{car.x:.1f}, {car.y:.1f}', xy=(car.x, car.y + 5), color='black', annotation_clip=False)

fargs = [
        Fargs(
            ax=ax,
            map_size_x=map_size_x,
            map_size_y=map_size_y,
            car=car,
            car_outline=car_outline,
            front_right_wheel=front_right_wheel,
            front_left_wheel=front_left_wheel,
            rear_right_wheel=rear_right_wheel,
            rear_left_wheel=rear_left_wheel,
            rear_axle=rear_axle,
            annotation=annotation,
            target=target
        )
    ]

ani = FuncAnimation(fig, animate, frames=frames, init_func=lambda: None, fargs=fargs, interval=interval, repeat=loop)
    # anim.save('animation.gif', writer='imagemagick', fps=50)
    
plt.grid()
plt.show()

# f = "/Users/theresejoseph/Documents/Neural_Network_Playground/results/GIFs/BerlinPathFollowing.gif" 
# writergif = animation.PillowWriter(fps=fps) 
# ani.save(f, writer=writergif)


In [125]:
outfile='../../results/testEnvPathVelocities.npy'
np.save(outfile,[velocity_x, velocity_y])

x=[0]
y=[0]

for i in range(len(velocity_x)):
    x.append(x[-1]+velocity_x[i])
    y.append(y[-1]+velocity_y[i])

plt.plot(x,y)
plt.axis('equal')


(-3.0026738082673647, 63.05614997361466, -4.04169830317138, 84.87566436659898)

In [118]:
plt.plot(velocity_x, 'b-')
plt.title('Linear x Velocity from Berlin Path Following Simulation')
plt.xlabel("Time [iterations]")
plt.ylabel("Velocity [m/s]")
plt.show()

In [9]:
plt.plot(velocity_y, 'g-')
plt.title('Linear y Velocity from Berlin Path Following Simulation')
plt.xlabel("Time [iterations]")
plt.ylabel("Velocity [m/s]")
plt.show()

In [10]:
plt.plot(angular_velocity, 'm-')
plt.title('Angular Velocity from Berlin Path Following Simulation')
plt.xlabel("Time [iterations]")
plt.ylabel("Velocity [rad/s]")
plt.show()

In [11]:
'''Pure Persuit'''
def control(q, R, speed, path, gainW):
# % Inputs:
# %  q is a 1x3 vector giving the current configuration of the robot in units of metres and radians
# %  R is the pure pursuit following distance
# %  speed is the forward speed of the vehicle
# %  path is an Nx2 matrix containing the path as a set of waypoints, columns are x- and y-coordinates respectively.
# % Return:
# %  vel is a 1x2 vector containing the requested velocity and turn rate of the robot [v, omega]

# %closest point 
    x, y = zip(*path)
    dist=np.zeros((len(path)))
    for i in range(1,len(path)):
        dist[i]=np.sqrt((x[i]-q[0])**2+(y[i]-q[1])**2)

    index=np.argmin(dist)
    C = [x[index], y[index]]
   
  
    for i in range(1,len(path)):
        dist2path=np.sqrt((C[0]-x[i])**2+(C[1]-y[i])**2)
        if dist2path > R: 
            g_index = i
            break

        if 'g_index' not in locals():
          g_index  = len(path)

    
    G=[x[g_index], y[g_index]]
    
# % controller to move from point to intersection    
    thetagoal=math.atan2((G[1]-q[1]),(G[0]-q[0]))
    W=gainW*(angdiff(q[2], thetagoal))
    
    d=np.sqrt((x[-1]-q[0])**2+(y[-1]-q[1])**2)
    
    vel=[speed, W]
    
    return vel 

def qupdate(q, vel, dt):
    # % Inputs:
    # % q is the configuration vector (x, y, theta) in units of metres and radians
    # % vel is the velocity vector (v, omega)
    # % dt is the length of the integration timestep in units of seconds
    # % Return:
    # % qnew is the new configuration vector vector (x, y, theta) in units of metres and radians at the
    # % end of the time interval.
    vel=np.array(vel)
    q=np.array(q)

    theta_old=q[2]
    xdot=vel[0]*np.cos(theta_old)
    ydot=vel[0]*np.sin(theta_old)
    thetadot=vel[1]

    x=dt*xdot+ q[0]
    y= dt*ydot + q[1]
    theta= dt*thetadot + q[2]

    qnew=[x, y, theta]

    return qnew

def drive2point(q,p):
    V=np.sqrt((p[0]-q[0])**2+(p[1]-q[1])**2)
    thetagoal=math.atan2((p[1]-q[1]),(p[0]-q[0]))
    W=angdiff(q[2], thetagoal)
    qnew=[p[0], p[1], thetagoal]
    return [V,W], qnew

q = [a[0], b[0], 0]
R = 0.1
speed = 0.15
gainW=0.3
vel = control(q, R, speed, path, gainW)
dt=0.2
# x,y=q[0],q[1]

roboX=[]
roboY=[]
velocities=[]
for i in range(1,len(path)):
    # q=qupdate(q, vel, dt)
    # vel = control(q, R, speed, path, gainW)
    vel, q=drive2point(q, [a[i], b[i]])
    roboX.append(q[0])
    roboY.append(q[1])
    velocities.append(vel[1])

# plt.imshow(img, cmap='gray')
# plt.plot(roboX,roboY)
# plt.plot(velocities)
# plt.show()
   

In [12]:
MAX_STEPS=5000
def moveDirection(x,y,direction):
    dic = {'west':(-1,0),'east':(1,0),'north':(0,-1),'south':(0,1),'northeast':(1,-1),'southeast':(-1,1),\
        'southwest':(-1,1),'northwest':(-1,-1)}
    x,y = x + dic[direction][0], y + dic[direction][1]
    return x,y

def follow_line(image, start_x, start_y, direction):
    # Initialize the current position
    x = start_x
    y = start_y

    # Initialize a list to store the path
    path = []

    # Keep track of the number of steps
    num_steps = 0

    # Loop until the line ends or the maximum number of steps is reached
    while num_steps < MAX_STEPS:
        # Add the current position to the path
        path.append((x, y))

        allDirections=["north","northeast","east","southeast", "south", "southwest","west","northwest"]
        random.shuffle(allDirections)
        new_x,new_y=moveDirection(x,y,direction)
        if image[new_y][new_x] == 1:
            # print(f"hit an edge{x,y} going {direction}")
            for dir in allDirections:
                new_x,new_y=moveDirection(x,y,dir)
                if image[new_y][new_x] == 0:
                    x=new_x
                    y=new_y
                    direction=dir
                    # print(f"found new direction {direction}")
                    break
        else:
            x=new_x
            y=new_y

        # # Check if the current position is still on the line
        # if image[y][x] == 1:
        #   # The line has ended, so break out of the loop
        #   break


        
        # Increment the number of steps
        num_steps += 1
    
  # Return the path
    return path

# path=follow_line(img, 45, 488, "northeast")